## 가져올 정보
1. 코디셋 이미지
2. 태그
2. 코디 아이템별 내용
    - 제품명
    - 브랜드
    - 좋아요
    - 제품 이미지

In [1]:
import pandas as pd
import numpy as np
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import re
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome()

capabilities = driver.capabilities

data_list = []

def item_collector(codi_url):
    driver.get(codi_url)
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 코디 상품 리스트 (여기부터 수정!!!!!!!!!!!!!!!)
    codi_item = driver.find_elements(By.XPATH, '//a[@class="styling_img"]')

    # 각 a 태그에서 href 속성 값 추출하여 data_no 리스트에 추가
    data_no_list = []
    for a_tag in codi_item:
        href_value = a_tag.get_attribute("href")
        data_no_list.append(int(href_value.split("/")[-1]))

    items_list = []
    for d in data_no_list:
        product_url = f"https://www.musinsa.com/app/goods/{d}"
        driver.get(product_url)

        # Create a new soup object with the updated HTML content
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        item = {}
        item['big_category'] = soup.find('a', class_='product-detail__sc-up77yl-1 hYzMBL').text if soup.find('a', class_='product-detail__sc-up77yl-1 hYzMBL') else None
        print(item['big_category'])

        # 중분류 수정해야함!
        item['mid_category'] = driver.find_element(By.XPATH, '//*[@id="product-top"]/div/div[1]/a[2]').text
        print(item['mid_category'])

        item['product_name'] = soup.find('h3', class_='product-detail__sc-1klhlce-3 fjguJZ').text if soup.find('h3', class_='product-detail__sc-1klhlce-3 fjguJZ') else None
        print(item['product_name'])

        item['brand'] = soup.find('a', class_='product-detail__sc-achptn-9 dWHdxq').text if soup.find('a', class_='product-detail__sc-achptn-9 dWHdxq') else None
        print(item['brand'])

        # sex_xpath = ['//*[@id="product-right-top"]/div[1]/ul/li[2]/div[2]/span',
        #     '//*[@id="product-right-top"]/div[2]/ul/li[2]/div[2]/span[3]',
        #     '//*[@id="product-right-top"]/div[1]/ul/li[2]/div[2]/span[3]']
        
        # sex = None  # Initialize sex variable

        # for xpath in sex_xpath:
        #     try:
        #         sex_element = driver.find_element(By.XPATH, xpath)
        #         sex_text = sex_element.text
        #         break  # If '여성' is found, exit the loop
        #     except NoSuchElementException:
        #         pass

        # item['sex'] = sex

        # item['sex'] = driver.find_element(By.CSS_SELECTOR, '#product-right-top > div.product-detail__sc-achptn-0.eXRtIE > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.gfoaTb > span:nth-child(3)').text
        # print(item['sex'])
        item['like'] = soup.find('span', class_='product-detail__sc-achptn-4 flUHrZ').text if soup.find('span', class_='product-detail__sc-achptn-4 flUHrZ') else None
        print(item['like'])

        try:
            item['price'] = soup.find('span', class_='product-detail__sc-1p1ulhg-7 gQybFn').text if soup.find('span', class_='product-detail__sc-1p1ulhg-7 gQybFn') else None
        except NoSuchElementException:
            item['price'] = soup.find('span', class_ = 'product-detail__sc-1p1ulhg-7 kpTJXD').text if soup.find('span', class_='product-detail__sc-1p1ulhg-7 kpTJXD') else None
        print(item['price'])

        item['product_img'] = driver.find_element(By.XPATH, '//*[@id="bigimg"]').get_attribute('src')
        print(item['product_img'])
    
        items_list.append(item)
        time.sleep(3)

    return items_list
        
    



In [2]:
import pandas as pd
import numpy as np
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import re
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome()

capabilities = driver.capabilities

data_list = []

def musinsa_collector(url):
    driver.get(url)
    time.sleep(2)
    
    driver.find_element(By.CSS_SELECTOR, '#footerCommonPc > div.sc-pvystx-0.iRAZYo > button:nth-child(3)').click()
    time.sleep(2) 
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 코디 리스트 가져오기
    codi_list = []
    codi_list = [tag['onclick'].split("'")[1] for tag in soup.find_all('a', {'class': 'style-list-item__link'})]
    time.sleep(5)
    print(codi_list)

    # 코디 개별 url 들어가기
    for i in codi_list:
        codi_url = f'https://www.musinsa.com/app/codimap/views/{i}?style_type=americancasual&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1'
        driver.get(codi_url)

        codi = {}

        # 코디셋 이미지
        codi['codi_img'] = driver.find_element(By.XPATH, '//img[@class="photo"]').get_attribute('src')
        
        # 태그
        tag = driver.find_elements(By.XPATH, '//*[@id="style_info"]/div[1]/div/a')
        # 태그 텍스트 추출하여 리스트에 저장
        codi['tags'] = [tag.text for tag in tag]

        # 코디 상품 리스트 (여기부터 수정!!!!!!!!!!!!!!!)
        codi_item = driver.find_elements(By.XPATH, '//a[@class="styling_img"]')

        # 각 a 태그에서 href 속성 값 추출하여 data_no 리스트에 추가
        data_no_list = []
        for a_tag in codi_item:
            href_value = a_tag.get_attribute("href")
            data_no_list.append(int(href_value.split("/")[-1]))
        time.sleep(5)
        
        # 코디 아이템별 내용    
        items_list = item_collector(codi_url)
        data_list.extend(items_list)
        data_list.append(codi)
    print(codi)

page = 1
# 아메카지
url = 'https://www.musinsa.com/app/codimap/lists?style_type=americancasual&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page='

while page <= 1:
    new_url = url + str(page)
    musinsa_collector(new_url)
    print(page, '완료') # 완료 여부
    page += 1

df = pd.DataFrame(data_list)
csv_file_path = 'sample.csv'
df.to_csv(csv_file_path, index = False, encoding = 'utf-8')

driver.quit()



['29874', '29815', '29769', '29759', '29723', '29676', '29630', '29621', '29582', '29570', '29560', '29522', '29479', '29425', '29293', '29271', '29256', '29213', '29064', '29052', '28978', '28974', '28966', '28961', '28956', '28909', '28855', '28810', '28751', '28747', '28739', '28701', '28696', '28659', '28652', '28648', '28601', '28583', '28549', '28496', '28490', '28440', '28435', '28425', '28389', '28371', '28336', '28330', '28328', '28285', '28275', '28224', '28173', '28167', '28161', '28119', '28075', '28069', '28058', '28026']
아우터
숏패딩/숏헤비 아우터
더플 다운
GUESS
2,457
178,000원
https://image.msscdn.net/images/goods_img/20231016/3630442/3630442_16974445794370_500.jpg
상의
맨투맨/스웨트셔츠
마가렛 스웨트셔츠 - 오트밀
OURHOPE
5,740
72,000원
https://image.msscdn.net/images/goods_img/20230209/3070468/3070468_16759031955428_500.jpg
상의
셔츠/블라우스
C 로고 옥스포드 스트라이프 셔츠 라이트 블루
COVERNAT
6,551
79,000원
https://image.msscdn.net/images/goods_img/20200814/1550595/1550595_16794479149140_500.jpg
바지
코튼 팬츠
sea rover pants(womens) be

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   big_category  328 non-null    object
 1   mid_category  328 non-null    object
 2   product_name  328 non-null    object
 3   brand         328 non-null    object
 4   like          328 non-null    object
 5   price         195 non-null    object
 6   product_img   328 non-null    object
 7   codi_img      60 non-null     object
 8   tags          60 non-null     object
dtypes: object(9)
memory usage: 27.4+ KB


In [5]:
df.head(10)

,big_category,mid_category,product_name,brand,like,price,product_img,codi_img,tags
0,아우터,숏패딩/숏헤비 아우터,더플 다운,GUESS,"2,457","178,000원",https://image.msscdn.net/images/goods_img/2023...,NaN,NaN
1,상의,맨투맨/스웨트셔츠,마가렛 스웨트셔츠 - 오트밀,OURHOPE,"5,740","72,000원",https://image.msscdn.net/images/goods_img/2023...,NaN,NaN
2,상의,셔츠/블라우스,C 로고 옥스포드 스트라이프 셔츠 라이트 블루,COVERNAT,"6,551","79,000원",https://image.msscdn.net/images/goods_img/2020...,NaN,NaN
3,바지,코튼 팬츠,sea rover pants(womens) beige,UNIFORM BRIDGE WOMEN,38,"129,000원",https://image.msscdn.net/images/goods_img/2023...,NaN,NaN
4,여성 가방,크로스백,마마스 백 라지 -브라운(가죽),TUETTOI,383,None,https://image.msscdn.net/images/goods_img/2023...,NaN,NaN
5,양말/레그웨어,양말,AUTHENTIC SOCKS(AUTHENTIC ORANGE),FREI,88,"9,000원",https://image.msscdn.net/images/goods_img/2021...,NaN,NaN
6,신발,구두,1461 3홀 빈티지브라운 / 11838201,DR.MARTENS,"6,084","210,000원",https://image.msscdn.net/images/goods_img/2018...,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://image.msscdn.net/images/codimap/detail...,"[#겨울, #아메카지, #게스, #구두, #닥터마틴, #맨투맨/스웨트셔츠, #메신저..."
8,상의,니트/스웨터,SIOT4057 cowichan handmade sweater_Navy,SIYAZU,325,"327,000원",https://image.msscdn.net/images/goods_img/2022...,NaN,NaN
9,상의,니트/스웨터,우먼스 플로리 와펜 케이블 카라 니트 아이보리,WHATITISNT,"1,055","99,000원",https://image.msscdn.net/images/goods_img/2023...,NaN,NaN
